In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from haversine import haversine
import re
import gc
from sklearn.metrics import mean_absolute_error

import lightgbm as lgb
import xgboost

## data load

In [21]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

## 데이터 전처리

In [22]:
# base_date
train_base_date_df = train.groupby(['base_date','road_name']).count()[['id']].reset_index()
test_base_date_df = test.groupby(['base_date','road_name']).count()[['id']].reset_index()

train_base_date_df.columns = ['base_date','road_name','base_date_cnt']
test_base_date_df.columns = ['base_date','road_name','base_date_cnt']

In [23]:
train = train.merge(train_base_date_df, how='inner', on=['base_date','road_name'])
test = test.merge(test_base_date_df, how='inner', on=['base_date','road_name'])

In [24]:
# 위경도로 거리 도출
%time
train["distance"] = train[["start_latitude", "start_longitude", "end_latitude", "end_longitude"]].apply(lambda x: haversine((x[0], x[1]), (x[2], x[3])), axis="columns")
test["distance"] = test[["start_latitude", "start_longitude", "end_latitude", "end_longitude"]].apply(lambda x: haversine((x[0], x[1]), (x[2], x[3])), axis="columns")

Wall time: 0 ns


In [25]:
# label encoding 
str_col = ['day_of_week','start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [26]:
# one-hot
cols = ['day_of_week','road_type', 'start_turn_restricted', 'end_turn_restricted', 'lane_count', 'road_rating'
       ,'weight_restricted'] 
train = pd.get_dummies(train, prefix_sep='_', sparse=False, drop_first=True, columns = cols)
test = pd.get_dummies(test, prefix_sep='_', sparse=False, drop_first=True, columns = cols)

In [27]:
# del 
del_cols = ['id','height_restricted','vehicle_restricted', 'start_node_name', 'end_node_name', 'base_date', 'road_name']
train = train.drop(del_cols, axis=1)
test = test.drop(del_cols, axis=1)

## 모델링

In [28]:
gc.collect()

80

In [29]:
print(train.columns)

Index(['base_hour', 'multi_linked', 'connect_code', 'maximum_speed_limit',
       'start_latitude', 'start_longitude', 'end_latitude', 'end_longitude',
       'target', 'base_date_cnt', 'distance', 'day_of_week_1', 'day_of_week_2',
       'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6',
       'road_type_3', 'start_turn_restricted_1', 'end_turn_restricted_1',
       'lane_count_2', 'lane_count_3', 'road_rating_106', 'road_rating_107',
       'weight_restricted_32400.0', 'weight_restricted_43200.0',
       'weight_restricted_50000.0'],
      dtype='object')


train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [30]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(train.drop('target', axis=1) , train['target'], \
                                                    test_size=0.2, shuffle=True, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3760973, 26), (940244, 26), (3760973,), (940244,))

In [ ]:
%%time 
model = lgb.LGBMRegressor(random_state=42)

In [34]:
model = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7, random_state=42)

In [ ]:
%%time
model.fit(X_train, y_train)

## 예측

In [33]:
mean_absolute_error(y_test, model.predict(X_test))

4.4645174939292

In [ ]:
4.46

## 제출

In [36]:
sample_submission = pd.read_csv('input/sample_submission.csv')

In [39]:
%%time
model2 = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7, random_state=42)

model2.fit(train.drop('target', axis=1), train['target'])

[18:28:41] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\gbm\gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
Wall time: 10min 21s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.08, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.75,
             tree_method='approx', validate_parameters=1, verbosity=None)

In [40]:
sample_submission['target'] = model2.predict(test)

In [41]:
sample_submission.to_csv("output/20221023-2.csv", index = False)